In [0]:
!pip install seaborn --upgrade

Requirement already up-to-date: seaborn in /usr/local/lib/python3.6/dist-packages (0.9.0)


In [0]:
import numpy as np
import io
from google.colab import files
import pandas as pd

In [0]:
uploaded = files.upload()

Saving test_set_clean_only_tags.txt to test_set_clean_only_tags.txt
Saving test_set_clean_only_text.txt to test_set_clean_only_text.txt
Saving training_set_clean_only_tags.txt to training_set_clean_only_tags.txt
Saving training_set_clean_only_text.txt to training_set_clean_only_text.txt


In [0]:
!curl -o cc.pl.300.bin.gz https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pl.300.bin.gz
!gunzip cc.pl.300.bin.gz
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4294M  100 4294M    0     0  38.7M      0  0:01:50  0:01:50 --:--:-- 20.6M
cc.pl.300.bin  sample_data


In [0]:
with open('training_set_clean_only_text.txt', 'r') as f:
  train_text = f.readlines()
with open('test_set_clean_only_text.txt', 'r') as f:
  test_text = f.readlines()
train_tags = np.loadtxt('training_set_clean_only_tags.txt')

In [0]:
# baseline - losowanie klasy na podstawie ich częstości w zbiorze uczącym
freq0 = len(data_tags[data_tags==0])/len(data_tags)
print('Stosunek klasy 0:', freq0)
out = ['0\n' if np.random.uniform() < freq0 else '1\n' for i in range(len(test_text))]
with open('results.txt', 'w') as f:
  f.writelines(out)
files.download('results.txt')

Stosunek klasy 0: 0.915247485310228


In [0]:
from gensim.models import FastText

model = FastText.load_fasttext_format('cc.pl.300')

In [0]:
from gensim.utils import tokenize

def text2Vectors(text, vecLen = 300):
  sentVecs = np.zeros((len(text), vecLen))
  for i in range(len(text)):
    tokens = list(tokenize(text[i], lowercase = True))   #wywala liczby
    vec = np.zeros(vecLen)
    cnt = 0
    for token in tokens:
      if token == 'anonymized_account':
        continue
      try:
        vec += model.wv[token]
        cnt += 1
      except KeyError:
        continue
    sentVecs[i,] = vec / cnt if cnt else np.zeros(vecLen)
  return sentVecs

sentVecsTrain = text2Vectors(train_text)

In [0]:
from sklearn import svm

svmModel = svm.SVC(class_weight = {0:0.1, 1:0.9}, C = 1000, gamma='auto')
svmModel.fit(sentVecsTrain, train_tags)

SVC(C=1000, cache_size=200, class_weight={0: 0.1, 1: 0.9}, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
sentVecsTest = text2Vectors(test_text)
preds = svmModel.predict(sentVecsTest)
np.savetxt('results.txt', preds, fmt='%d')
files.download('results.txt')